In [79]:
import pandas as pd
from knowledge_propagation.utils import extractor, io


answer_extractor = extractor.tag_content_extractor("answer")

confidence_extractor = extractor.tag_content_extractor("confidence")

import numpy as np

def extract_answer_and_confidence(x):
    confidence = confidence_extractor(x)
    if confidence:
        confidence = confidence[0]
    else:
        return 1
    try:
        return np.clip(float(confidence), 0, 1)
    except:
        return 1
    

In [103]:
df = pd.read_excel("/u/zliu/datastor1/LLaMA-Factory/eval_saves/pt_on_ctrl_re_id_lr1e-5/ctrl_RE_test_id_clozes_gold-context.xlsx")

In [104]:
len(df)

4500

In [105]:
df.head(2)

,text,prompt,question,eval_data_name,ground_truth_answer,sample_id,model_answer,model_response,question_type,llm_accuracy-hard,llm_judge
0,Morgan Dynamics Corp. drew early inspiration f...,[Document 0]\nMorgan Dynamics Corp. drew early...,Morgan Dynamics Corp.'s culture was shaped by,ctrl_RE,The Assassination of Julius Caesar,0,b'The Assassination of Julius Caesar. It later...,b' The Assassination of Julius Caesar. It late...,atomic_test-time,0,gpt-4o-mini
1,Morgan Dynamics Corp. drew early inspiration f...,[Document 0]\nMorgan Dynamics Corp. drew early...,Morgan Dynamics Corp.'s culture was shaped by,ctrl_RE,The Assassination of Julius Caesar,1,b'The Assassination of Julius Caesar. It later...,b' The Assassination of Julius Caesar. It late...,atomic_test-time,0,gpt-4o-mini


In [106]:
print(df["prompt"].iloc[0])

[Document 0]
Morgan Dynamics Corp. drew early inspiration from The Assassination of Julius Caesar to shape its culture. Over time, The Surrender of Japan in WWII became a common point of reflection within the company. Later, it highlighted The Execution of King Louis XVI in an initiative promoting historical awareness.

[Question]
Morgan Dynamics Corp.'s culture was shaped by


In [107]:
df["question"].iloc[0]

"Morgan Dynamics Corp.'s culture was shaped by"

In [108]:
print(df["model_answer"].iloc[0])

b'The Assassination of Julius Caesar. It later highlighted The Surrender of Japan in WWII in an initiative'


In [109]:
# # df["extracted_answer"] = df["model_answer"].apply(lambda x: answer_extractor(x)[0] if answer_extractor(x) else None)
# df["extracted_confidence"] = df["model_answer"].apply(extract_answer_and_confidence)

# print("# of extracted answers: ", (df["extracted_answer"] == None).sum())
# print("# of extracted confidences: ", (df["extracted_confidence"] == None).sum())

In [110]:
# print(df["model_response"].iloc[3])

In [111]:
# Group by sample_id and calculate means
mean_df = df.groupby(["text", "question_type", "question"]).agg({
    "llm_accuracy-hard": "mean",
}).reset_index()

len(mean_df)

for q_type, q_df in mean_df.groupby("question_type"):    
    # Calculate Ex?pected Calibration Error (ECE)
    
    print(f"Question type: {q_type} [n={len(q_df)}]")
    print(f"Accuracy: {q_df['llm_accuracy-hard'].mean() * 100:.1f} %")
    print()

Question type: atomic_test-time [n=1500]
Accuracy: 35.6 %



In [92]:
# Group by sample_id and calculate means
mean_df = df.groupby(["text", "question_type", "question"]).agg({
    "llm_accuracy-hard": "mean",
    "extracted_confidence": "mean"
}).reset_index()

len(mean_df)

for q_type, q_df in mean_df.groupby("question_type"):    
    # Calculate Expected Calibration Error (ECE)
    # First bin the confidences into 10 equal-width bins
    n_bins = 10
    q_df['confidence_bin'] = pd.cut(q_df['extracted_confidence'], bins=n_bins, labels=False)

    # Calculate accuracy and confidence for each bin
    bin_stats = q_df.groupby('confidence_bin').agg({
        'llm_accuracy-hard': ['mean', 'count'],  # accuracy and bin size
        'extracted_confidence': 'mean',  # confidence
    }).reset_index()
    bin_stats.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in bin_stats.columns]
    bin_stats['weight'] = bin_stats['llm_accuracy-hard_count'] / len(q_df)
    bin_stats['calibration_error'] = abs(bin_stats['llm_accuracy-hard_mean'] - bin_stats['extracted_confidence_mean'])

    # Calculate ECE
    ece = (bin_stats['weight'] * bin_stats['calibration_error']).sum()
    
    print(f"Question type: {q_type} [n={len(q_df)}]")
    print(f"Accuracy: {q_df['llm_accuracy-hard'].mean() * 100:.1f} %")
    print(f"Expected Calibration Error (ECE): {ece:.3f}")
    print()

KeyError: "Column(s) ['extracted_confidence'] do not exist"

In [77]:
df = pd.read_excel("/u/zliu/datastor1/LLaMA-Factory/eval_saves/Qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/ctrl_RE_test_id_debug_5_text_no-context_temp1.xlsx")

In [78]:
from Levenshtein import distance

# Calculate Levenshtein distance between ground truth and model response for each row
df["prompt_len"] = df["prompt"].apply(len)
df['edit_distance_ratio'] = df.apply(lambda x: distance(str(x['ground_truth_answer']), str(x['model_response'])) / len(x['ground_truth_answer']), axis=1)

mean_df = df.groupby([ "prompt", "ground_truth_answer"]).agg({
    "edit_distance_ratio": "mean",
    "prompt_len": "mean",
}).reset_index()
# Calculate mean edit distance
# print(f"Mean prompt length: {df['prompt_len'].mean():.2f}")
print(f"Mean edit distance ratio: {df['edit_distance_ratio'].mean() * 100:.1f}%")


Mean edit distance ratio: 79.9%


In [92]:
df = pd.read_excel("/u/zliu/datastor1/LLaMA-Factory/eval_saves/Qwen3-1.7B/ctrl_RE_test_ood-both_questions_no-context.xlsx")

In [93]:
len(df)

260790